In [0]:
import re
from nltk.util import ngrams

In [0]:
# import pandas and read file
import pandas as pd
df = pd.read_csv('tweets.csv')

In [0]:
# tweets for which the status column is not empty is a labelled tweet
df_labelled_tweets = df[df['status'].notnull()]

In [0]:
# function to create ngrams of all labelled tweets: ignore numbers, empty tokens and short tokens
output_list = []

def labelled_ngrams(num_gram):
  for s in df_labelled_tweets['report']:
    s = s.lower()
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    tokens = [token for token in s.split(" ") if token != "" and len(token) > 3 and token.isnumeric() == False ]
    output = list(ngrams(tokens, num_gram))
    output_list.append(output)
  return output_list

In [0]:
# confirm that trigrams were created for all 390 labelled tweets
output = labelled_ngrams(3)
len(output)

In [0]:
# function to find tweets that match a given trigram
def find_matches(ls):
  
  found_matches = []

  # select batch of tweets to process
  for i,s in df[40000:60000].iterrows():

    # get the column containing tweets
    tweet = s['report']

    # some tweets are NaNs, empty or contain just a number, skip them
    if isinstance(tweet, float) == True:
      continue
    
    # else prepare tweet for matching
    else:
      tweet = ''.join([str(i) for i in tweet if not i.isdigit()])
    
    # if tweet contains all 3 component words of a trigram, we will save it if it has not been saved before
    res = all(ele in tweet for ele in ls)
    if i not in found_matches and res == True:
      found_matches.append(i)
  return found_matches

In [0]:
# count to show what's going on as you wait
k = len(output)
i = 0

all_matches = []
matches = []

# for each tweet
for li in output:
  print('Now processing tweet {} of {}'.format(i,k))

  # for each trigram in the tweet
  for ls in li:
    
    # find tweet ids that match the trigram and save them
    matches = find_matches(ls)
  i = i+1
  all_matches.append(matches)

In [0]:
# get all the returned ids by all the labelled tweet trigram search in a 1-d list, remove potential duplicate ids
match_ids = []
for item in all_matches:
  for id in item:
    if id not in match_ids and id != '':
      match_ids.append(id)

In [0]:
# select all tweets from the main df with selected ids
df_selected = df[df.index.isin(match_ids)]

In [0]:
# one last check to remove duplicate tweets; they sometime creep in
df_nodups_selected = df_select.drop_duplicates(subset=['report'], keep='first')

In [0]:
# remember to change x in file name to match batch name
df_nodups_selected.to_csv('match_labelled_tweets_x.csv', index=False)